In [4]:
# Демонстрація нового LLM-підходу без embeddings/clustering
from app.tools.youtube import fetch_comments
from app.tools.preprocess import select_fast_batch, preprocess_comments_df
from app.tools.topics_taxonomy import TAXONOMY, ID2NAME
from app.tools.topics_llm import classify_llm_full, aggregate_topics, sample_quotes

# 1) Завантаження та підготовка даних
df_all = fetch_comments("https://www.youtube.com/watch?v=26riTPNOJbc",
                       sqlite_path="./.cache.db",
                       include_replies=True,
                       max_comments=5000)

df_fast = select_fast_batch(df_all, mode="top_likes", limit=1200, include_replies=False)

df_pre = preprocess_comments_df(
    df_fast,
    min_chars=12
    # keep_langs=None - залишаємо всі мови для LLM аналізу
)

print(f"Завантажено: {len(df_all)} коментарів")
print(f"Відібрано: {len(df_fast)} коментарів")
print(f"Після препроцесингу: {len(df_pre)} коментарів")
print(df_pre[["comment_id","text_clean","lang","like_count"]].head())


Завантажено: 102 коментарів
Відібрано: 57 коментарів
Після препроцесингу: 44 коментарів
                   comment_id  \
0  Ugx8YMgNr70XWO9_PGN4AaABAg   
1  UgxITgSgyx1a2BgJHd94AaABAg   
2  UgwnH4Az1RliZfk5B7x4AaABAg   
3  UgyPwsZ3OWS21CNg0AF4AaABAg   
4  UgziduOc_8k-xFGd80F4AaABAg   

                                          text_clean lang  like_count  
0  Чернівці люблю, чекаю на наступне відео) Вітаю...   uk          17  
1  Щоб відчути справжню Україну у часи війни вам ...   uk          12  
2  W końcu doczekałem się nowego odcinka. Pozdrawiam   pl          11  
3  Kowalski,nie pieprz bzdury,bo u was parkany ta...   pl          11  
4  Ale filmiki robisz na poziomie. Bravo. A wypra...   pl          10  


In [7]:
# 2) LLM-класифікація через OpenRouter
# Увага: потрібен OPENROUTER_API_KEY в .env файлі

import os

print("\n🧠 Демонстрація LLM-класифікації...")
print(f"Таксономія містить {len(TAXONOMY)} категорій:")
for t in TAXONOMY:
    print(f"  • {t['id']}: {t['name']}")

# Перевіряємо API ключ
if not os.getenv("OPENROUTER_API_KEY"):
    print("\n⚠️  OPENROUTER_API_KEY не знайдено!")
    print("💡 Для запуску потрібно:")
    print("   1. Створити .env файл з ключем: OPENROUTER_API_KEY=your_key")
    print("   2. Або встановити змінну середовища")
    print("\n🔍 Показую структуру даних замість класифікації:")
    
    # Створюємо mock результати для демонстрації
    df_classified = df_pre.copy()
    df_classified["topic_labels_llm"] = [["praise"] if i % 3 == 0 else ["questions"] if i % 3 == 1 else ["suggestions"] for i in range(len(df_pre))]
    df_classified["topic_top_llm"] = [labels[0] if labels else None for labels in df_classified["topic_labels_llm"]]
    
    print("📊 Mock структура результатів:")
    display(df_classified[["comment_id", "text_clean", "topic_labels_llm", "topic_top_llm"]].head())
    
else:
    print("\n✅ API ключ знайдено, запускаємо класифікацію...")
    
    # Класифікуємо коментарі (можна зменшити batch_size для економії)
    df_classified = classify_llm_full(
        df_pre, 
        TAXONOMY, 
        text_col="text_clean", 
        batch_size=5  # зменшений batch для демо
    )

    print(f"\n✅ Класифіковано {len(df_classified)} коментарів")

    # Аналіз результатів
    top_topics = aggregate_topics(df_classified).head(5)
    print(f"\n🏆 Топ-{len(top_topics)} тем:")

    for i, (_, row) in enumerate(top_topics.iterrows(), 1):
        topic_name = ID2NAME.get(row['topic_id'], row['topic_id'])
        print(f"{i}. {topic_name}: {int(row['count'])} ({row['share']*100:.1f}%)")
        
        # Показуємо найкращі цитати
        quotes = sample_quotes(df_classified, row["topic_id"], k=2)
        for quote in quotes:
            text = (quote["text"] or "")[:100].replace("\n", " ")
            print(f"   💬 {text}...")

    print(f"\n📊 Результати збережено в df_classified")
    
    # 🆕 Перевіряємо, чи збереглися результати в БД
    from app.tools.classification_db import get_topic_statistics
    
    # Витягуємо video_id з df_classified
    video_id = df_classified['video_id'].iloc[0] if 'video_id' in df_classified.columns else "26riTPNOJbc"
    stats = get_topic_statistics(video_id, "./.cache.db")
    
    if stats['total_comments'] > 0:
        print(f"✅ Збережено в БД: {stats['total_comments']} класифікованих коментарів")
    else:
        print("⚠️  Результати НЕ збережено в БД. Спробуємо зберегти вручну...")
        
        # Додаємо video_id якщо його немає
        if 'video_id' not in df_classified.columns:
            df_classified['video_id'] = "26riTPNOJbc"
        
        # Зберігаємо вручну
        from app.tools.classification_db import save_classification_results
        success = save_classification_results(df_classified, "./.cache.db")
        
        if success:
            print("✅ Результати успішно збережено в БД!")
            stats = get_topic_statistics(video_id, "./.cache.db")
            print(f"   Збережено: {stats['total_comments']} коментарів")
        else:
            print("❌ Помилка збереження в БД")
    
    display(df_classified[["comment_id", "text_clean", "topic_labels_llm", "topic_top_llm"]].head())



🧠 Демонстрація LLM-класифікації...
Таксономія містить 11 категорій:
  • praise: Похвала/подяка
  • critique: Критика/незадоволення
  • questions: Питання/уточнення
  • suggestions: Поради/пропозиції
  • host_persona: Ведучий/персона
  • content_truth: Точність/правдивість
  • av_quality: Звук/відео/монтаж
  • price_value: Ціни/цінність
  • personal_story: Особисті історії
  • offtopic_fun: Офтоп/жарти/меми
  • toxicity: Токсичність/хейт

✅ API ключ знайдено, запускаємо класифікацію...
Overall: 11.1% | Chunk: 1/9 | Remaining: 16.5s
Overall: 22.2% | Chunk: 2/9 | Remaining: 14.4s
Overall: 33.3% | Chunk: 3/9 | Remaining: 12.2s
Overall: 44.4% | Chunk: 4/9 | Remaining: 10.0s
Overall: 55.6% | Chunk: 5/9 | Remaining: 8.7s
Overall: 66.7% | Chunk: 6/9 | Remaining: 6.1s
Overall: 77.8% | Chunk: 7/9 | Remaining: 3.9s
Overall: 88.9% | Chunk: 8/9 | Remaining: 2.0s
Overall: 100.0% | Chunk: 9/9 | Remaining: 0.0s

✅ Класифіковано 44 коментарів

🏆 Топ-5 тем:
1. Критика/незадоволення: 14 (28.0%)
   💬 Kow

,comment_id,text_clean,topic_labels_llm,topic_top_llm
0,Ugx8YMgNr70XWO9_PGN4AaABAg,"Чернівці люблю, чекаю на наступне відео) Вітаю...",[praise],praise
1,UgxITgSgyx1a2BgJHd94AaABAg,Щоб відчути справжню Україну у часи війни вам ...,[suggestions],suggestions
2,UgwnH4Az1RliZfk5B7x4AaABAg,W końcu doczekałem się nowego odcinka. Pozdrawiam,[praise],praise
3,UgyPwsZ3OWS21CNg0AF4AaABAg,"Kowalski,nie pieprz bzdury,bo u was parkany ta...",[critique],critique
4,UgziduOc_8k-xFGd80F4AaABAg,Ale filmiki robisz na poziomie. Bravo. A wypra...,[praise],praise


In [8]:
# 3) Перевірка збережених результатів та демонстрація CLI функцій

print("🔍 Перевірка збережених результатів у БД...")

from app.tools.classification_db import (
    get_video_list_with_classification, 
    load_classification_results,
    get_topic_statistics
)

# Список всіх відео в БД
videos_df = get_video_list_with_classification("./.cache.db")
print(f"\n📋 Відео в базі даних: {len(videos_df)}")

if not videos_df.empty:
    print("\nСписок проаналізованих відео:")
    for _, row in videos_df.iterrows():
        coverage = f"{row['classification_coverage']:.1f}%" if row['classification_coverage'] > 0 else "0%"
        print(f"  • {row['video_id']}: {row['classified_comments']}/{row['total_comments']} ({coverage})")

# Статистика для нашого відео
video_id = "26riTPNOJbc"
stats = get_topic_statistics(video_id, "./.cache.db")

print(f"\n📊 Статистика для відео {video_id}:")
print(f"   Всього коментарів: {stats['total_in_db']}")
print(f"   Класифіковано: {stats['total_comments']}")
print(f"   Покриття: {stats['classification_coverage']}%")

if stats['topics']:
    print("\n🏆 Топ-3 теми (з БД):")
    for i, topic in enumerate(stats['topics'][:3], 1):
        topic_name = ID2NAME.get(topic['topic'], topic['topic'])
        print(f"   {i}. {topic_name}: {topic['count']} ({topic['share_percent']}%)")

print(f"\n💡 Тепер можна використовувати CLI команди:")
print(f"   python -m app.analyze_llm show {video_id}")
print(f"   python -m app.analyze_llm list")
print(f"   python -m app.analyze_llm clear --video {video_id}")


🔍 Перевірка збережених результатів у БД...

📋 Відео в базі даних: 1

Список проаналізованих відео:
  • 26riTPNOJbc: 44/102 (43.1%)

📊 Статистика для відео 26riTPNOJbc:
   Всього коментарів: 102
   Класифіковано: 44
   Покриття: 43.1%

🏆 Топ-3 теми (з БД):
   1. Похвала/подяка: 13 (29.5%)
   2. Ціни/цінність: 7 (15.9%)
   3. Критика/незадоволення: 7 (15.9%)

💡 Тепер можна використовувати CLI команди:
   python -m app.analyze_llm show 26riTPNOJbc
   python -m app.analyze_llm list
   python -m app.analyze_llm clear --video 26riTPNOJbc
